## Imports 

In [1]:
import argparse
import easydict
import numpy as np
import numpy
import pandas as pd
import tensorflow as tf
from matplotlib import pyplot as plt
from tensorflow.keras.models import load_model
from plotread import *
from collections import Counter
%matplotlib inline

from omnixai.data.tabular import Tabular
from omnixai.preprocessing.base import Identity
from omnixai.preprocessing.tabular import TabularTransform
from omnixai.explainers.tabular import TabularExplainer
from omnixai.visualization.dashboard import Dashboard
from omnixai.data.timeseries import Timeseries
from omnixai.explainers.timeseries import TimeseriesExplainer
from omnixai.explainers.timeseries import ShapTimeseries
from omnixai.explainers.data import DataAnalyzer
from omnixai.explainers.tabular import ALE
from omnixai.explainers.tabular import CounterfactualExplainer
from omnixai.explainers.tabular.agnostic.L2X.l2x import L2XTabular
from omnixai.explainers.tabular import PartialDependenceTabular

C:\Users\User\anaconda3\envs\omnixai_env\lib\site-packages\shap\utils\_clustering.py:35: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  def _pt_shuffle_rec(i, indexes, index_mask, partition_tree, M, pos):
C:\Users\User\anaconda3\envs\omnixai_env\lib\site-packages\shap\utils\_clustering.py:54: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  def delta_minimization_or

In [2]:
from tensorflow.compat.v1.keras.backend import get_session
tf.compat.v1.disable_v2_behavior()                          # does not work for thesis, except with DeepExplainer
tf.compat.v1.disable_eager_execution()                    # does not work

Instructions for updating:
non-resource variables are not supported in the long term


## Functions 

In [3]:
def get_data():
    ###########################################################################
    # Parser Definition
    ###########################################################################
    opt = easydict.EasyDict({
        "model": "GRU",
        "datapath": "Dataset1/",
        "savepath": "Experiment1/GRU/8/Run1/",
        "extension": ".dat",
        "batch_size": 32,
        "plots_in": False,
        "plots_out": True
    })

    ###########################################################################
    # Variables Definition
    ###########################################################################
    nin = ['time', 'PLML2', 'PLMR', 'AVBL', 'AVBR']
    nout = ['time', 'DB1', 'LUAL', 'PVR', 'VB1']
    neurons = ['time','DB1','LUAL','PVR','VB1','PLML2','PLMR','AVBL','AVBR']

    ###########################################################################
    # Read data
    ###########################################################################
    files = getdata(opt.datapath, opt.extension)
    train, valid, test = splitdata(files)
    trainx, trainy = readdata(opt.datapath, train, neurons, nin, nout)
    validx, validy = readdata(opt.datapath, valid, neurons, nin, nout)
    testx, testy = readdata(opt.datapath, test, neurons, nin, nout)
    if opt.plots_in:
        plotdata(trainx, '/train_data', '/x', opt.model, opt.savepath)
        plotdata(trainy, '/train_data', '/y', opt.model, opt.savepath)
        plotdata(validx, '/valid_data', '/x', opt.model, opt.savepath)
        plotdata(validy, '/valid_data', '/y', opt.model, opt.savepath)
        plotdata(testx, '/test_data', '/x', opt.model, opt.savepath)
        plotdata(testy, '/test_data', '/y', opt.model, opt.savepath)
        
    ###########################################################################
    # Load Model and Evaluate
    ###########################################################################
    output_size = 4
    global model
    model = load_model(opt.savepath + 'model.h5')
    model.summary()

    X_train = np.array(trainx)
    X_test = np.array(testx)
    nin_names = nin[1:len(nin)]
        
    return X_train, X_test, nin_names

In [37]:
def custom_predictor_shaptimeseries(X):
    
    print(X.shape)
    print(type(X))
    X = np.array(X.values)
    print(X.shape)
    print(type(X))
    X = np.reshape(X, (10, 1000, 4))
    print(X.shape)
    print(type(X))
    
    predictions = model.predict(X)
    print(predictions.shape)
    print(type(predictions))
#     predictions_reshaped = np.reshape(predictions, (5000, 4000), order='F')
    
#     return predictions_reshaped
    predictions = np.reshape(predictions, (10000, 4), order='F')
    print(predictions.shape)
    print(type(predictions))
    predictions = Timeseries(predictions)
    print(predictions.shape)
    print(type(predictions))
    return predictions

In [ ]:
def custom_predictor(X):
    print(X.shape)
    print(type(X))
    predictions = model.predict(X)
#     predictions_reshaped = np.reshape(predictions, (5000, 4000), order='F')
    print(predictions.shape)
    print(type(predictions))
    return predictions
#     return predictions_reshaped

## Code 

In [5]:
X_train, X_test, nin_names = get_data()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 gru_1 (GRU)                 multiple                  312       
                                                                 
 time_distributed_1 (TimeDis  multiple                 36        
 tributed)                                                       
                                                                 
Total params: 348
Trainable params: 348
Non-trainable params: 0
_________________________________________________________________


### Timeseries

#### TimeseriesExplainer

In [ ]:
X_train_reshaped = np.reshape(X_train, (20, 4000), order='F')

In [ ]:
X_train_Timeseries = Timeseries(X_train[0])
# X_train_Timeseries = Timeseries(X_train_reshaped)

In [ ]:
X_test_reshaped = np.reshape(X_test, (10, 4000), order='F')

In [ ]:
X_test_Timeseries = Timeseries(X_test[0])
# X_test_Timeseries = Timeseries(X_test_reshaped)

In [ ]:
X_train_Timeseries.shape

In [ ]:
X_test_Timeseries.shape

In [28]:
explainers = TimeseriesExplainer(
    explainers=["shap"],
    mode="forecasting",
    data=X_train_Timeseries,
    model=custom_predictor
)

NameError: name 'custom_predictor' is not defined

In [ ]:
explainers.explain(X_test[0])

#### ShapTimeseries

In [25]:
X_train_reshaped = np.reshape(X_train, (20000, 4), order='F')
# X_train_reshaped = np.reshape(X_train, (20, 4000), order='F')

In [26]:
# X_train_Timeseries = Timeseries(X_train[0])
# X_train_Timeseries = Timeseries(X_train)
X_train_Timeseries = Timeseries(X_train_reshaped)

In [38]:
# explainers = ShapTimeseries(
#     training_data=X_train_Timeseries,
#     predict_function=model.predict,
#     mode='forecasting'
# )
explainers = ShapTimeseries(
    training_data=X_train_Timeseries,
    predict_function=custom_predictor_shaptimeseries,
    mode='forecasting'
)

In [30]:
# X_test_reshaped = np.reshape(X_test, (10, 4000), order='F')
X_test_reshaped = np.reshape(X_test, (10000, 4), order='F')

In [31]:
# X_test_Timeseries = Timeseries(X_test[0])
X_test_Timeseries = Timeseries(X_test_reshaped)

In [27]:
X_train_Timeseries.shape

(20000, 4)

In [32]:
X_test_Timeseries.shape

(10000, 4)

In [13]:
X_test_Timeseries = X_test_Timeseries[:1000]

In [14]:
X_test_Timeseries.shape

(1000, 4)

In [39]:
# explainers.explain(X_test_Timeseries[0])
explainers.explain(X_test_Timeseries)

(10000, 4)
<class 'omnixai.data.timeseries.Timeseries'>
(10000, 4)
<class 'numpy.ndarray'>
(10, 1000, 4)
<class 'numpy.ndarray'>


`Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.


(10, 1000, 4)
<class 'numpy.ndarray'>
(10000, 4)
<class 'numpy.ndarray'>
(10000, 4)
<class 'omnixai.data.timeseries.Timeseries'>
(10000, 4)
<class 'omnixai.data.timeseries.Timeseries'>
(10000, 4)
<class 'numpy.ndarray'>
(10, 1000, 4)
<class 'numpy.ndarray'>
(10, 1000, 4)
<class 'numpy.ndarray'>
(10000, 4)
<class 'numpy.ndarray'>
(10000, 4)
<class 'omnixai.data.timeseries.Timeseries'>
(10000, 4)
<class 'omnixai.data.timeseries.Timeseries'>
(10000, 4)
<class 'numpy.ndarray'>
(10, 1000, 4)
<class 'numpy.ndarray'>
(10, 1000, 4)
<class 'numpy.ndarray'>
(10000, 4)
<class 'numpy.ndarray'>
(10000, 4)
<class 'omnixai.data.timeseries.Timeseries'>
(10000, 4)
<class 'omnixai.data.timeseries.Timeseries'>
(10000, 4)
<class 'numpy.ndarray'>
(10, 1000, 4)
<class 'numpy.ndarray'>
(10, 1000, 4)
<class 'numpy.ndarray'>
(10000, 4)
<class 'numpy.ndarray'>
(10000, 4)
<class 'omnixai.data.timeseries.Timeseries'>
(10000, 4)
<class 'omnixai.data.timeseries.Timeseries'>
(10000, 4)
<class 'numpy.ndarray'>
(10, 1

(10, 1000, 4)
<class 'numpy.ndarray'>
(10000, 4)
<class 'numpy.ndarray'>
(10000, 4)
<class 'omnixai.data.timeseries.Timeseries'>
(10000, 4)
<class 'omnixai.data.timeseries.Timeseries'>
(10000, 4)
<class 'numpy.ndarray'>
(10, 1000, 4)
<class 'numpy.ndarray'>
(10, 1000, 4)
<class 'numpy.ndarray'>
(10000, 4)
<class 'numpy.ndarray'>
(10000, 4)
<class 'omnixai.data.timeseries.Timeseries'>
(10000, 4)
<class 'omnixai.data.timeseries.Timeseries'>
(10000, 4)
<class 'numpy.ndarray'>
(10, 1000, 4)
<class 'numpy.ndarray'>
(10, 1000, 4)
<class 'numpy.ndarray'>
(10000, 4)
<class 'numpy.ndarray'>
(10000, 4)
<class 'omnixai.data.timeseries.Timeseries'>
(10000, 4)
<class 'omnixai.data.timeseries.Timeseries'>
(10000, 4)
<class 'numpy.ndarray'>
(10, 1000, 4)
<class 'numpy.ndarray'>
(10, 1000, 4)
<class 'numpy.ndarray'>
(10000, 4)
<class 'numpy.ndarray'>
(10000, 4)
<class 'omnixai.data.timeseries.Timeseries'>
(10000, 4)
<class 'omnixai.data.timeseries.Timeseries'>
(10000, 4)
<class 'numpy.ndarray'>
(10, 1

(10, 1000, 4)
<class 'numpy.ndarray'>
(10000, 4)
<class 'numpy.ndarray'>
(10000, 4)
<class 'omnixai.data.timeseries.Timeseries'>
(10000, 4)
<class 'omnixai.data.timeseries.Timeseries'>
(10000, 4)
<class 'numpy.ndarray'>
(10, 1000, 4)
<class 'numpy.ndarray'>
(10, 1000, 4)
<class 'numpy.ndarray'>
(10000, 4)
<class 'numpy.ndarray'>
(10000, 4)
<class 'omnixai.data.timeseries.Timeseries'>
(10000, 4)
<class 'omnixai.data.timeseries.Timeseries'>
(10000, 4)
<class 'numpy.ndarray'>
(10, 1000, 4)
<class 'numpy.ndarray'>
(10, 1000, 4)
<class 'numpy.ndarray'>
(10000, 4)
<class 'numpy.ndarray'>
(10000, 4)
<class 'omnixai.data.timeseries.Timeseries'>
(10000, 4)
<class 'omnixai.data.timeseries.Timeseries'>
(10000, 4)
<class 'numpy.ndarray'>
(10, 1000, 4)
<class 'numpy.ndarray'>
(10, 1000, 4)
<class 'numpy.ndarray'>
(10000, 4)
<class 'numpy.ndarray'>
(10000, 4)
<class 'omnixai.data.timeseries.Timeseries'>
(10000, 4)
<class 'omnixai.data.timeseries.Timeseries'>
(10000, 4)
<class 'numpy.ndarray'>
(10, 1

Creating an ndarray from nested sequences exceeding the maximum number of dimensions of 32 is deprecated. If you mean to do this, you must specify 'dtype=object' when creating the ndarray.


ValueError: operands could not be broadcast together with shapes (1000000,) (100,) 

### Tabular

In [ ]:
X_train_reshaped = np.reshape(X_train, (20, 4000), order='F')

In [ ]:
X_train_tabular = Tabular(X_train_reshaped)

In [ ]:
print(X_train_tabular.shape)

In [ ]:
transformer = TabularTransform(
    target_transform=Identity()
).fit(X_train_tabular)

In [ ]:
# Initialize a TabularExplainer
explainers = TabularExplainer(
    explainers=["lime", "sensitivity", "pdp", "ale"],
    mode="regression",
    data=X_train_tabular,
    model=custom_predictor,
    preprocess=lambda z: transformer.transform(z)
)

In [ ]:
X_test_reshaped = np.reshape(X_test, (10, 4000), order='F')

In [ ]:
X_test_tabular = Tabular(X_test_reshaped)

In [ ]:
print(X_test_tabular.shape)

In [ ]:
explainers.explain(X_test_tabular)

#### DataAnalyzer

In [ ]:
X_train_reshaped = np.reshape(X_train, (20000, 4), order='F')

In [ ]:
df = pd.DataFrame(X_train_reshaped, columns=nin_names)
print(df)

In [ ]:
tabular_data = Tabular(df)

In [ ]:
# Initialize a `DataAnalyzer` explainer.
# We can choose multiple explainers/analyzers by specifying analyzer names.
# In this example, the first explainer is for feature correlation analysis and
# the others are for feature imbalance analysis (the same explainer with different parameters).
explainer = DataAnalyzer(
    explainers=["correlation"],
    mode="regression",
    data=tabular_data
)

In [ ]:
explanations = explainer.explain()

In [ ]:
explanations["correlation"].ipython_plot()

#### ALE

In [ ]:
X_train_reshaped = np.reshape(X_train, (20000, 4), order='F')

In [ ]:
tabular_data = Tabular(
    X_train_reshaped,
    feature_columns=nin_names
)

In [ ]:
explainer = ALE(
    training_data=tabular_data,
    predict_function=custom_predictor_shaptimeseries
)

In [ ]:
explanations = explainer.explain()

In [ ]:
explanations.ipython_plot()

#### Counterfactuals

In [ ]:
X_train_reshaped = np.reshape(X_train, (20000, 4), order='F')

In [ ]:
X_test_reshaped = np.reshape(X_test, (10000, 4), order='F')

In [ ]:
tabular_data = Tabular(
    X_train_reshaped,
    feature_columns=nin_names
)

In [ ]:
explainer = CounterfactualExplainer(
    training_data=tabular_data,
    predict_function=model
)

In [ ]:
explanations = explainer.explain(X_test_reshaped)

In [ ]:
explanations.ipython_plot()

#### L2X

In [ ]:
X_train_reshaped = np.reshape(X_train, (20000, 4), order='F')

In [ ]:
tabular_data = Tabular(
    X_train_reshaped,
    feature_columns=nin_names
)

In [ ]:
explainer = L2XTabular(
    training_data=tabular_data,
    predict_function=model
)

#### PDP

In [ ]:
X_train_reshaped = np.reshape(X_train, (20000, 4), order='F')

In [ ]:
tabular_data = Tabular(
    X_train_reshaped,
    feature_columns=nin_names
)

In [ ]:
explainer = PartialDependenceTabular(
    training_data=tabular_data,
    predict_function=model
)

In [ ]:
explanations = explainer.explain(
    features=nin_names)

In [ ]:
explanations.ipython_plot()